<font size="6"> Data Engineering Project — Retail Store Part 3 — Data Warehousing </font>

<font size="5"> Step 1 — Design the Data Warehouse — In Practice</font>

<font size="4"> Connecting Python to MySQL</font>

In [7]:
import numpy as np
import pandas as pd
import names
from faker import Faker
faker = Faker()
import pandasql as ps
import random
import time
from datetime import datetime
from datetime import date

In [8]:
connection = pymysql.connect(host ='localhost',port=int(3306),user='root',passwd='Qwerty1234/',db='dwh_whiskey')

# Creating a Cursor object
cursor = connection.cursor()

<font size="4"> Generating daily dates and creating a DataFrame</font>

In [9]:
start_date =date(1990,1,1)
end_date = date(2100,1,1)

# Get all the dates from 1990-01-01 to 2100-01-01
dates = pd.date_range(start=start_date, end=end_date)

dates_df = pd.DataFrame(dates, columns=['Date'])

# Insert into a dataframe

   <font size="4">Generating the date table </font>  

In [10]:
dates_df['Date_key'] =np.random.default_rng().choice(range(1,len(dates_df['Date'])+1), len(dates_df['Date']), replace = False)

# Day_name Column
dates_df['Day_name'] = dates.day_name()

# Month Column
month_dict = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',
             5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',
             11:'Nov',12:'Dec'}
dates_df['Month'] = dates_df.Date.dt.month
dates_df.Month.replace(month_dict,inplace=True)

# Year Column
dates_df['Year'] = dates_df.Date.dt.year

# Confirm that the Date_key column is unique
assert len(dates_df.Date_key.unique()) == len(dates_df.Date)

In [11]:
dates_df

,Date,Date_key,Day_name,Month,Year
0,1990-01-01,36222,Monday,Jan,1990
1,1990-01-02,20989,Tuesday,Jan,1990
2,1990-01-03,37243,Wednesday,Jan,1990
3,1990-01-04,19636,Thursday,Jan,1990
4,1990-01-05,37520,Friday,Jan,1990
...,...,...,...,...,...
40173,2099-12-28,37509,Monday,Dec,2099
40174,2099-12-29,24331,Tuesday,Dec,2099
40175,2099-12-30,21465,Wednesday,Dec,2099
40176,2099-12-31,26436,Thursday,Dec,2099


   <font size="4">Loading the Date Dimension to MySQL </font>

In [12]:
query = "CREATE TABLE dwh_date2 (Dates DATE NOT NULL,Date_key INT NOT NULL PRIMARY KEY,Day_name VARCHAR(50) NOT NULL,Month_name VARCHAR(50) NOT NULL,Year_name VARCHAR(50) NOT NULL);"

cursor.execute(query)

0

In [13]:
records = dates_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)


for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into dwh_date2 (Dates,Date_key, Day_name, Month_name, Year_name) values {}".format(result[data])
    
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()